In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import scipy.stats as stats
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/data-storm-10/credit_card_default_train.csv
/kaggle/input/data-storm-10/DATA STORM 1.0 - First Round Competition Guidlines.pdf
/kaggle/input/data-storm-10/credit_card_default_test.csv
/kaggle/input/data-storm-10/Credit_card_default - Business Problem - Assessment Criteria - Data Dictionary.xlsx


In [2]:
train_data = pd.read_csv('../input/data-storm-10/credit_card_default_train.csv')
test_data = pd.read_csv('../input/data-storm-10/credit_card_default_test.csv')

In [3]:
#train_data.dtypes.sample

<bound method NDFrame.sample of Client_ID             object
Balance_Limit_V1      object
Gender                object
EDUCATION_STATUS      object
MARITAL_STATUS        object
AGE                   object
PAY_JULY               int64
PAY_AUG                int64
PAY_SEP                int64
PAY_OCT                int64
PAY_NOV                int64
PAY_DEC                int64
DUE_AMT_JULY           int64
DUE_AMT_AUG            int64
DUE_AMT_SEP            int64
DUE_AMT_OCT            int64
DUE_AMT_NOV            int64
DUE_AMT_DEC            int64
PAID_AMT_JULY          int64
PAID_AMT_AUG           int64
PAID_AMT_SEP           int64
PAID_AMT_OCT           int64
PAID_AMT_NOV           int64
PAID_AMT_DEC           int64
NEXT_MONTH_DEFAULT     int64
dtype: object>

In [4]:
def New_balance(x):
    if x[-1]=='M':
        x=float(x[:-1])*1000000
    elif x[-1]=='K':
        x=float(x[:-1])*1000
    else:
        x=float(x)
    return(x)

In [5]:
train_data["Balance_Limit_V1_new"] = train_data["Balance_Limit_V1"].apply(New_balance)
train_data.drop("Balance_Limit_V1",axis=1,inplace=True)
train_data.rename(columns={"Balance_Limit_V1_new":"Balance_Limit_V1"},inplace=True)
train_col = list(train_data.columns)
last = train_col.pop()
train_col.insert(1,last)
train_data= train_data[train_col]

test_data["Balance_Limit_V1_new"] = test_data["Balance_Limit_V1"].apply(New_balance)
test_data.drop("Balance_Limit_V1",axis=1,inplace=True)
test_data.rename(columns={"Balance_Limit_V1_new":"Balance_Limit_V1"},inplace=True)
test_col = list(test_data.columns)
last = test_col.pop()
test_col.insert(1,last)
test_data= test_data[test_col]

In [6]:
gender={"M":0 ,"F":1} 
status ={"Single":0 , "Other":1} 
study= {"Graduate":0 , "High School":1 , "Other":2} 
age= {"Less than 30":0 , "31-45":1, "46-65":2 , "More than 65":3}

train_data.Gender = [gender[item] for item in train_data.Gender]
train_data.EDUCATION_STATUS = [study[item] for item in train_data.EDUCATION_STATUS]
train_data.MARITAL_STATUS = [status[item] for item in train_data.MARITAL_STATUS]
train_data.AGE=[age[item] for item in train_data.AGE]

In [7]:
test_data.Gender = [gender[item] for item in test_data.Gender]
test_data.EDUCATION_STATUS = [study[item] for item in test_data.EDUCATION_STATUS]
test_data.MARITAL_STATUS = [status[item] for item in test_data.MARITAL_STATUS]
test_data.AGE=[age[item] for item in test_data.AGE]

In [8]:
index = train_data.pop("Client_ID")
y = train_data.pop("NEXT_MONTH_DEFAULT")

index_test = test_data.pop("Client_ID")

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( train_data, y, test_size=0.3)

In [10]:
X_train=X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values
test_data = test_data.values

In [12]:
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
X_train1 = scX.fit_transform(np.concatenate((X_train[:,0:1],X_train[:,12:]),axis=1))
X_test1 = scX.transform(np.concatenate((X_test[:,0:1],X_test[:,12:]),axis=1))
test_data1 = scX.transform(np.concatenate((test_data[:,0:1],test_data[:,12:]),axis=1))
np.shape(X_train1)

(16800, 12)

In [13]:
X_train1 = np.concatenate((X_train1,X_train[:,1:12]),axis=1)
X_test1 = np.concatenate((X_test1,X_test[:,1:12]),axis=1)
test_data1 = np.concatenate((test_data1,test_data[:,1:12]),axis=1)

In [14]:
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error,classification_report


In [15]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train1, y_train)

y_pred = xgb_model.predict(X_test1)

print(confusion_matrix(y_test, y_pred))

[[5323  260]
 [1033  584]]


In [19]:
print(classification_report(y_test,y_pred,digits=4))

              precision    recall  f1-score   support

           0     0.8375    0.9534    0.8917      5583
           1     0.6919    0.3612    0.4746      1617

    accuracy                         0.8204      7200
   macro avg     0.7647    0.6573    0.6832      7200
weighted avg     0.8048    0.8204    0.7980      7200



In [17]:
prediction = xgb_model.predict(test_data1)

In [18]:
op = {'Client_ID':index_test,'NEXT_MONTH_DEFAULT':prediction}
df = pd.DataFrame(op,columns=['Client_ID','NEXT_MONTH_DEFAULT'])
export_csv=df.to_csv('prediction.csv',index=None,header=True)